In [1]:
import requests
from bs4 import BeautifulSoup
url = "https://www.opindia.com/latest-news/"
data = requests.get(url)

In [2]:
data

<Response [200]>

In [3]:
soup = BeautifulSoup(data.content,"html.parser")

In [4]:
def fetch_article(url):
    data = requests.get(url)
    soup = BeautifulSoup(data.content,"html.parser")
    articles = []
    for i in soup.find_all("h3",class_ = ["entry-title td-module-title"]):
        articles.append(i.find('a')['title'])     
    return articles

In [5]:
urllist = []
for i in range(2,21,1):
    url = "https://www.opindia.com/latest-news/page/" + str(i) + "/"
    urllist.append(url)

In [6]:
all_articles = []
for i in urllist:
    all_articles.extend(fetch_article(i))

In [7]:
all_articles

['India reaffirms at the UN that non-Abrahamic religions too face hate and malice amidst propaganda of ‘Islamophobia’ in India',
 'Agnipath recruitment violence: Meet Manish Bhaiya, who allegedly sent clueless ‘protestors’ to protest against the scheme',
 'BJP MP Sadhvi Pragya Singh Thakur receives death threat from man claiming to be working for Iqbal Kaskar, case registered in Bhopal',
 'National Conference leader Farooq Abdullah withdraws his name as joint opposition’s Presidential candidate',
 'Pawan Khera offered a lollipop of the communication department in Congress for his ‘tapasya’ after he whined about Rajya Sabha seat',
 'Terrorists join ‘toolkit’ gang after AltNews’ Mohammad Zubair triggers Islamists globally over Nupur Sharma’s remarks on Prophet Muhammad',
 'National Herald and the crooked history of Congress leadership',
 'As US condemns the statements by Nupur Sharma, here is what a Pakistani Maulana said about its hand in the global outcry',
 'The Nupur Sharma fiasco an

In [9]:
p_art =[]
for i in all_articles:
    q = i.upper()
    import re
    q = re.sub("[^A-Z0-9 ]","",q)
    from nltk.stem import PorterStemmer
    tk_q = q.split(" ")
    sent = ""
    for j in tk_q:
        ps = PorterStemmer()
        sent = sent + " " + ps.stem(j).upper()
    p_art.append(sent)

In [10]:
p_art

[' INDIA REAFFIRM AT THE UN THAT NONABRAHAM RELIGION TOO FACE HATE AND MALIC AMIDST PROPAGANDA OF ISLAMOPHOBIA IN INDIA',
 ' AGNIPATH RECRUIT VIOLENC MEET MANISH BHAIYA WHO ALLEGEDLI SENT CLUELESS PROTESTOR TO PROTEST AGAINST THE SCHEME',
 ' BJP MP SADHVI PRAGYA SINGH THAKUR RECEIV DEATH THREAT FROM MAN CLAIM TO BE WORK FOR IQBAL KASKAR CASE REGIST IN BHOPAL',
 ' NATION CONFER LEADER FAROOQ ABDULLAH WITHDRAW HI NAME AS JOINT OPPOSIT PRESIDENTI CANDID',
 ' PAWAN KHERA OFFER A LOLLIPOP OF THE COMMUN DEPART IN CONGRESS FOR HI TAPASYA AFTER HE WHINE ABOUT RAJYA SABHA SEAT',
 ' TERRORIST JOIN TOOLKIT GANG AFTER ALTNEW MOHAMMAD ZUBAIR TRIGGER ISLAMIST GLOBAL OVER NUPUR SHARMA REMARK ON PROPHET MUHAMMAD',
 ' NATION HERALD AND THE CROOK HISTORI OF CONGRESS LEADERSHIP',
 ' AS US CONDEMN THE STATEMENT BY NUPUR SHARMA HERE IS WHAT A PAKISTANI MAULANA SAID ABOUT IT HAND IN THE GLOBAL OUTCRI',
 ' THE NUPUR SHARMA FIASCO AND THE LIBER 9 WAY HOW THE ISLAMOFASCIST AND THEIR IDEOLOG COMRAD HAVE SHOT TH

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
A = tf.fit_transform(p_art).toarray()

In [12]:
A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=5)
cl_res = km.fit(A)

In [14]:
cl_res.labels_

array([3, 2, 0, ..., 4, 4, 4])

In [16]:
import pandas as pd
Q = pd.DataFrame(p_art,columns=["Article"])
Q['Cluster']=cl_res.labels_

In [17]:
E = {1 : "politics",
2 : "religion",
3 : "sports",
4 : "entertainment",
0 : "geopolitics"}

In [18]:
R = []
for i in Q.Cluster:
    R.append(E[i])

Q['category'] = R

In [19]:
Q

,Article,Cluster,category
0,INDIA REAFFIRM AT THE UN THAT NONABRAHAM RELI...,3,sports
1,AGNIPATH RECRUIT VIOLENC MEET MANISH BHAIYA W...,2,religion
2,BJP MP SADHVI PRAGYA SINGH THAKUR RECEIV DEAT...,0,geopolitics
3,NATION CONFER LEADER FAROOQ ABDULLAH WITHDRAW...,3,sports
4,PAWAN KHERA OFFER A LOLLIPOP OF THE COMMUN DE...,4,entertainment
...,...,...,...
1971,DELHI POLIC REACH RAJASTHAN DIRECT RAPEACCUS ...,4,entertainment
1972,WATCH ARM TALIBAN MEN DANC INSID CLASSROOM AF...,4,entertainment
1973,RAKESH TIKAIT OUST FROM BHARATIYA KISAN UNION...,4,entertainment
1974,UTTAR PRADESH ILLEG ARM FACTORI RUN NEAR IDGA...,4,entertainment
